抓三間公司的外資佔比

In [1]:
import requests as rq
import json as jsn
import pandas as pd
import glob as gb
import time
import os

單一一日資料抓取觀察測試

In [2]:
# 目標股票代號
ids = ["2357", "2382", "2417"]
# 日期
date: str = "20231212"
# 目標網址
url: str = f"https://www.twse.com.tw/rwd/zh/fund/MI_QFIIS?date={date}&selectType=25&response=json&_=1701913040002"
# response物件 (method = GET)
response_obj = rq.request(method="GET", url=url)
# response物件得到的的json字串
json_str: str = response_obj.text
# 以字串的方式讀取json
# 注意load跟loads 「s」的差別
# load讀「json『檔』」;loads讀「json『字串』」
# json讀出來是dict
dic: dict = jsn.loads(s=json_str)
dic

{'stat': 'OK',
 'date': '20231212',
 'selectType': '25',
 'title': '112年12月12日 外資及陸資投資持股統計',
 'hints': '單位:股',
 'fields': ['證券代號',
  '證券名稱',
  '國際證券編碼',
  '發行股數',
  '外資及陸資尚可投資股數',
  '全體外資及陸資持有股數',
  '外資及陸資尚可投資比率',
  '全體外資及陸資持股比率',
  '外資及陸資共用法令投資上限比率',
  '陸資法令投資上限比率',
  '與前日異動原因(註)',
  '最近一次上市公司申報外資及陸資持股異動日期'],
 'data': [['2301',
   '光寶科',
   'TW0002301009',
   '2,368,245,832',
   '1,498,285,412',
   '869,960,420',
   63.26,
   36.73,
   '100.00',
   '100.00',
   '',
   '112/11/17'],
  ['2305',
   '全友',
   'TW0002305000',
   '205,660,827',
   '204,921,069',
   '739,758',
   99.64,
   0.35,
   '100.00',
   '100.00',
   '',
   '112/04/10'],
  ['2324',
   '仁寶',
   'TW0002324001',
   '4,407,146,625',
   '2,398,316,332',
   '2,008,830,293',
   54.41,
   45.58,
   '100.00',
   '100.00',
   '',
   '112/05/22'],
  ['2331',
   '精英',
   'TW0002331006',
   '557,402,968',
   '462,057,844',
   '95,345,124',
   82.89,
   17.1,
   '100.00',
   '100.00',
   '',
   '112/05/23'],
  ['2352',
   '佳世達',
   

dic有以下的key，根據觀察，我們發現「欄」是'fields'這個key對應到的值；「所有列資料」是'data'這個key對應到的值；日期是'date'這個key對應到的值。

In [3]:
dic.keys()

dict_keys(['stat', 'date', 'selectType', 'title', 'hints', 'fields', 'data', 'notes', 'total'])

欄(column)

In [3]:
cols: list = dic["fields"]
cols

['證券代號',
 '證券名稱',
 '國際證券編碼',
 '發行股數',
 '外資及陸資尚可投資股數',
 '全體外資及陸資持有股數',
 '外資及陸資尚可投資比率',
 '全體外資及陸資持股比率',
 '外資及陸資共用法令投資上限比率',
 '陸資法令投資上限比率',
 '與前日異動原因(註)',
 '最近一次上市公司申報外資及陸資持股異動日期']

獲取列全部資料，是二維陣列。第一個維度裡面每一個元素是每一列資料，與欄對應。

In [5]:
datas:list = dic["data"]
datas

[['2301',
  '光寶科',
  'TW0002301009',
  '2,368,245,832',
  '1,498,285,412',
  '869,960,420',
  63.26,
  36.73,
  '100.00',
  '100.00',
  '',
  '112/11/17'],
 ['2305',
  '全友',
  'TW0002305000',
  '205,660,827',
  '204,921,069',
  '739,758',
  99.64,
  0.35,
  '100.00',
  '100.00',
  '',
  '112/04/10'],
 ['2324',
  '仁寶',
  'TW0002324001',
  '4,407,146,625',
  '2,398,316,332',
  '2,008,830,293',
  54.41,
  45.58,
  '100.00',
  '100.00',
  '',
  '112/05/22'],
 ['2331',
  '精英',
  'TW0002331006',
  '557,402,968',
  '462,057,844',
  '95,345,124',
  82.89,
  17.1,
  '100.00',
  '100.00',
  '',
  '112/05/23'],
 ['2352',
  '佳世達',
  'TW0002352002',
  '1,966,781,958',
  '1,722,158,893',
  '244,623,065',
  87.56,
  12.43,
  '100.00',
  '100.00',
  '',
  '112/05/24'],
 ['2353',
  '宏碁',
  'TW0002353000',
  '3,047,853,828',
  '1,827,248,799',
  '1,220,605,029',
  59.95,
  40.04,
  '100.00',
  '100.00',
  '',
  '112/06/01'],
 ['2356',
  '英業達',
  'TW0002356003',
  '3,587,475,066',
  '2,768,905,668',
  '8

找出要篩選的"證券代號"在cols中的index

In [4]:
id_index: int = cols.index("證券代號")
id_index

0

for迴圈寫法，一個陣列裡面的每一個元素是一維陣列，篩選條件是一維陣列index 0的值是在target_ids裡面的

In [6]:
target_datas: list = [data for data in datas if data[id_index] in ids]
target_datas

[['2357',
  '華碩',
  'TW0002357001',
  '742,760,280',
  '322,233,235',
  '420,527,045',
  43.38,
  56.61,
  '100.00',
  '100.00',
  " <a href=' https://mops.twse.com.tw/server-java/t98sb03?step=1&TYPEK=sii&colorchg=1&kind=3&co_id=2357&date=11212' target=_blank>4</a>",
  '112/06/02'],
 ['2382',
  '廣達',
  'TW0002382009',
  '3,862,627,432',
  '2,955,214,838',
  '907,412,594',
  76.5,
  23.49,
  '100.00',
  '100.00',
  '',
  '112/05/18'],
 ['2417',
  '圓剛',
  'TW0002417003',
  '158,068,864',
  '154,317,764',
  '3,751,100',
  97.62,
  2.37,
  '100.00',
  '100.00',
  '',
  '112/08/31']]

產生一天的dataframe

In [7]:
df_test = pd.DataFrame(data=target_datas, columns=cols)
df_test

,證券代號,證券名稱,國際證券編碼,發行股數,外資及陸資尚可投資股數,全體外資及陸資持有股數,外資及陸資尚可投資比率,全體外資及陸資持股比率,外資及陸資共用法令投資上限比率,陸資法令投資上限比率,與前日異動原因(註),最近一次上市公司申報外資及陸資持股異動日期
0,2357,華碩,TW0002357001,"742,760,280","322,233,235","420,527,045",43.38,56.61,100.00,100.00,<a href=' https://mops.twse.com.tw/server-jav...,112/06/02
1,2382,廣達,TW0002382009,"3,862,627,432","2,955,214,838","907,412,594",76.50,23.49,100.00,100.00,,112/05/18
2,2417,圓剛,TW0002417003,"158,068,864","154,317,764","3,751,100",97.62,2.37,100.00,100.00,,112/08/31


將日期字串(YYYYMMDD)處理成(YYYY-MM-DD)

In [8]:
date:str = dic["date"]
# str:   YYYYMMDD
# index: 01234567
# date[:4] + "-" + date[4:6] + "-" + date[6:]
#   YYYY   + "-" +     DD    + "-" +   "MM"
date = date[:len("YYYY")] + "-" + date[len("YYYY"):len("YYYYMM")] + "-" + date[len("YYYYMM"):]
# L = [0]
# L * 3
# >> [0, 0, 0]
dates: list = [date] * df_test.shape[0]
dates

['2023-12-12', '2023-12-12', '2023-12-12']

新增一欄為「日期」的資料 資料是用剛剛處理好的日期 這樣就知道這是哪一天的資料

In [9]:
df_test["日期"] = dates
df_test

,證券代號,證券名稱,國際證券編碼,發行股數,外資及陸資尚可投資股數,全體外資及陸資持有股數,外資及陸資尚可投資比率,全體外資及陸資持股比率,外資及陸資共用法令投資上限比率,陸資法令投資上限比率,與前日異動原因(註),最近一次上市公司申報外資及陸資持股異動日期,日期
0,2357,華碩,TW0002357001,"742,760,280","322,233,235","420,527,045",43.38,56.61,100.00,100.00,<a href=' https://mops.twse.com.tw/server-jav...,112/06/02,2023-12-12
1,2382,廣達,TW0002382009,"3,862,627,432","2,955,214,838","907,412,594",76.50,23.49,100.00,100.00,,112/05/18,2023-12-12
2,2417,圓剛,TW0002417003,"158,068,864","154,317,764","3,751,100",97.62,2.37,100.00,100.00,,112/08/31,2023-12-12


選一個沒有開盤的日子(假日)看看，len(dic["data"]) == 0 或 dic["total"] = 0，之後要爬的時候如果遇到就跳過

In [10]:
date = "20190101"
url: str = f"https://www.twse.com.tw/rwd/zh/fund/MI_QFIIS?date={date}&selectType=25&response=json&_=1701913040002"
response_obj = rq.request(method="GET", url=url)
json_str: str = response_obj.text
dic: dict = jsn.loads(s=json_str)
print(f"len(dic['data']) = {len(dic['data'])}")
print("=" * 50)
dic

len(dic['data']) = 0


{'stat': 'OK',
 'date': '20190101',
 'selectType': '25',
 'title': '108年01月01日 外資及陸資投資持股統計',
 'fields': ['證券代號',
  '證券名稱',
  '國際證券編碼',
  '發行股數',
  '外資及陸資尚可投資股數',
  '全體外資及陸資持有股數',
  '外資及陸資尚可投資比率',
  '全體外資及陸資持股比率',
  '外資及陸資共用法令投資上限比率',
  '陸資法令投資上限比率',
  '與前日異動原因(註)',
  '最近一次上市公司申報外資及陸資持股異動日期'],
 'data': [],
 'notes': "一、與前日異動原因以空白及數字2,3,4,5表示，其代表意義說明如下：\n'空白'係指外陸資於集中市場交易所產生之股數異動，不包含外陸資配認股數、外資原始股東出售股數、存託憑證發行等異動。\n  '2'係上市公司因股本變動申報最近一次除權交易日或股東會外陸資持股情形、因依規定之固定基準日申報外陸資持股情形、因前次申報資料有誤予以更新、或因發行海外有價證券、合併、公開收購等申報外陸資持股情形。\n  '3'係指非前項因素所產生之持股變動，如海外存託憑證經證期局核可尚未發行之預扣等。\n  '4'係依海外存託憑證流通餘額及外國人將海外存託憑證兌回普通股或將普通股再發行海外存託憑證情形更新外陸資持股數。\n  '5'係依上市公司每月申報國內海外有價證券轉換情形更新外陸資持股數。\n二、ETF證券代號第六碼為K、M、S、C者，表示該ETF以外幣交易。",
 'total': 0}

定義函式

In [11]:
# 代入想爬的股票代號陣列還有日期 之後跑回圈用日期迭代會用到
# 只是把之前爬一天的資料的過程套進來 不贅述
def get_df(ids: list, date: str):
    sauce_url: str = f"https://www.twse.com.tw/rwd/zh/fund/MI_QFIIS?date={date}&selectType=25&response=json&_=1701913040002"
    response_obj = rq.request(method="GET", url=sauce_url)
    json_str: str = response_obj.text
    dic: dict = jsn.loads(s=json_str)
    if len(dic["data"]) == 0:
        return
    id_index: int = dic["fields"].index("證券代號")
    target_datas: list = [data for data in dic["data"] if data[id_index] in ids]
    df = pd.DataFrame(data=target_datas, columns=dic["fields"])
    df["日期"] = [dic["date"][:len("YYYY")] + "-" + dic["date"][len("YYYY"):len("YYYYMM")] + "-" + dic["date"][len("YYYYMM"):]] * df.shape[0]
    return df

# 根據輸入的年、月參數 判斷那個月的最後一天為幾號 ex:閏年2月最後一天是29日
def get_month_end_day(year: int, month: int):
    # 如果是2月
    if month == 2:
        # 根據判斷是否為閏年回傳28或29
        if year % 4 == 0 and not(year % 100 == 0) or year % 400 == 0:
            return 29
        else:
            return 28
    # 如果是有31天的月份
    elif month in [1, 3, 5, 7, 8, 10, 12]:
        return 31
    # 其他如果是有30天的月份
    else:
        return 30

執行迴圈

In [20]:
ids = ["2357", "2382", "2417"]

# 一次執行爬一年的資料量。
# 因為怕快爬完前發生網路斷線或伺服器回應愈時等等的狀況而功虧一簣，我們決定每個月輸出一次csv。
# 就算發生狀況中斷，我們只要看斷在哪個月，把for month in range()的開頭調成斷掉的那個月繼續跑就好。
# 所以我們會先迭代月份，每個月會有一個dfs(儲存dataframe的陣列)儲存當月每天的dataframe。
# 然後在當月那圈迴圈結束前concat在一起，形成一個一整個月的每日資料。
# 當月的每日資料輸出成csv
year = 2019
for month in range(1, 12 + 1):
    dfs: list = []
    for day in range(1, get_month_end_day(year=year, month=month) + 1):
        date = f"{year}{'0' if month < 10 else ''}{month}{'0' if day < 10 else ''}{day}"
        df_day = get_df(ids=ids, date=date)
        
        # 依照上面get_df()這個函式所提到的，如果當天是假日或國定假日或過年之類的...
        # 函式會直接return一個None空值，如果當天dataframe回傳回來不是空值，就append進當月的dfs
        # 沒有的話就不執行。
        if df_day is not None:
            dfs.append(df_day)
        
        # 讓程式暫停，避免爬取資料間隔過短，也就是request頻率太高而被誤認成DDoS攻擊，所以休息1秒(0.25 * 4)，順便做一個進度條動畫。
        p = 4
        for i in range(p):
            print(f"剛爬完{date}的資料 休息中" + ("." * i + (" " * (p - i))), end="\r")
            time.sleep(0.25)
    
    # 將dfs裡面的每日資料concat成一個csv，參數「axis=0」執行列合併。
    df_month = pd.concat(dfs, axis=0, ignore_index=True)
    # 把一個月的每日資料輸出成csv，輸出到invest_month_datas這個資料夾。
    df_month.to_csv(path_or_buf=f"{os.getcwd()}\\invest_month_datas\\invest_{year}{'0' if month < 10 else ''}{month}.csv", encoding="UTF-8", index=False)
    
print(f"完成{year}年的資料爬取               ", end="\r")

將整理好的csv整理輸出成三個股票的csv

In [15]:
# 將2019年1月~2023年4月，共52個月的52個csv讀取並合併
files = gb.glob(pathname=f"{os.getcwd()}\\invest_month_datas\\*.csv", recursive=False)
dfs: list = []

# 作法同上，先將每個csv讀成dataframe並先append進dfs(dataframe的陣列)。
for file in files:
    file_csv = pd.read_csv(filepath_or_buffer=file, encoding="UTF-8")
    df = pd.DataFrame(data=file_csv)
    dfs.append(df)

# 最後再concat在一起。
df_all = pd.concat(objs=dfs, axis=0)
# 根據迴圈的特殊寫法分出3個股票的dataframe並放入dfs陣列。
dfs = [df_all.iloc[[i for i in range(df_all.shape[0]) if str(df_all.iloc[i]["證券代號"]) == id]] for id in ids]

# 輸出三個股票五年來的每日資料csv到invest_datas這個資料夾，共3個。
for df in dfs:
    df.to_csv(path_or_buf=f"{os.getcwd()}\\invest_datas\\invest_{df.iloc[0]['證券代號']}.csv", sep=",", encoding="UTF-8", index=False)